In [1]:
import uproot
from uproot_methods import TLorentzVectorArray
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob

In [2]:
YEAR = 2018
SAMPLE = "vbshwwlvlvbb_c2v"
# SAMPLE = "bosons.root"

NOMINAL_SAMPLE = f"../hadds/v2.6_SS/v2/createMini_{YEAR}/{SAMPLE}.root"
JEC_UP_SAMPLE = f"../hadds/v2.6_SS/v2_jecUp/createMini_{YEAR}/{SAMPLE}.root"
JEC_DN_SAMPLE = f"../hadds/v2.6_SS/v2_jecDn/createMini_{YEAR}/{SAMPLE}.root"

RUN2_NOMINAL_SAMPLE = f"../hadds/v2.6_SS/v2/createMini_Run2/{SAMPLE}.root"

In [3]:
def get_systs(df, wgt_col_pattern, sr_col_pattern, nom_wgt):
    rows = []
    for SR in df.columns[df.columns.str.contains(sr_col_pattern)]:
        for wgt_col in df.columns[df.columns.str.contains(wgt_col_pattern)]:
            in_SR = df[SR]
            n_pass_wgt = np.sum(df[in_SR][nom_wgt]*df[in_SR][wgt_col])
            n_pass_nom = np.sum(df[in_SR][nom_wgt])
            rows.append({
                "region":SR.split("is_")[-1],
                "wgt_name": wgt_col,
                "n_pass_wgt": n_pass_wgt,
                "n_pass_nom": n_pass_nom,
                "delta_percent": (1 - n_pass_wgt/(n_pass_nom))*100
            })
            
    return pd.DataFrame(data=rows)

def make_syst_table(df, wgt_col_pattern, sr_col_pattern, nominal_wgt="", debug=False):
    # Compute all systematics
    systs = get_systs(df, wgt_col_pattern, sr_col_pattern, nominal_wgt)
    if debug:
        display(systs)
    # Designate how columns should be renamed
    renames = {
        "delta_percent": wgt_col_pattern.replace("_", ""), 
        "region": "systematics"
    }
    # Return only one row: max systematic for each SR
    return (systs.groupby("region", as_index=False)[["delta_percent"]]
                 .agg(func=lambda x: np.max(np.abs(x)))
                 .round(1)
                 .rename(columns=renames)
                 .set_index("systematics")
                 .transpose())

def append_systs(syst_table, new_syst):
    if np.any(np.isin(new_syst.index, syst_table.index)):
        print("WARNING: this systematic is already in the table. No action taken.")
        return syst_table
    else:
        return syst_table.append(new_syst)

In [4]:
with uproot.open(NOMINAL_SAMPLE) as f:
    df = f.get("variable").pandas.df(
        branches=["is_ps*", "is_bdt_*", "is_cut_*", "LHE_*", "pu_rewgt*",
                  "lepsf*", "btagsf*", "trigsf*", "wgt", "genrewgt"]
    )
# Compute nominal event weight
df["evt_wgt_nominal"] = df.wgt*df.genrewgt*df.lepsf*df.btagsf*df.trigsf*df.pu_rewgt
# Alter up/down variations of scale factors s.t. we can apply them as follows:
# (event weight up) = (nominal event weight)*(sf up/down)
for sf in ["lepsf", "btagsf", "trigsf", "pu_rewgt"]:
    df[f"{sf}_up"] /= df[sf]
    df[f"{sf}_dn"] /= df[sf]

df[df.columns[df.columns.str.contains("is_")]] = df[df.columns[df.columns.str.contains("is_")]].astype(bool)

df

,is_ps_el,is_ps_mu,is_ps_tau,is_ps_neg,is_ps_lgt,is_ps,is_bdt_sr_el,is_bdt_sr_mu,is_bdt_sr_tau,is_bdt_sr_neg,...,lepsf_dn,btagsf,btagsf_up,btagsf_dn,trigsf,trigsf_up,trigsf_dn,wgt,genrewgt,evt_wgt_nominal
entry,,,,,,,,,,,,,,,,,,,,,
0,False,True,False,False,True,True,False,True,False,False,...,0.990358,0.900174,1.037809,0.962779,0.980000,1.01,0.99,0.001013,1.0,0.000876
1,False,True,False,False,True,True,False,True,False,False,...,0.997811,0.878428,1.071929,0.930369,0.980000,1.01,0.99,0.001013,1.0,0.000865
2,True,False,False,False,True,True,True,False,False,False,...,0.961429,0.903295,1.036638,0.963921,1.000000,1.01,0.99,0.001013,1.0,0.000793
3,True,False,False,False,True,True,True,False,False,False,...,0.975533,0.916477,1.040425,0.960188,1.000000,1.01,0.99,0.001013,1.0,0.000781
4,False,False,True,False,False,True,False,False,True,False,...,0.946379,0.900039,1.019932,0.980265,0.960853,1.05,0.95,0.001013,1.0,0.000791
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3801,False,True,False,False,True,True,False,False,False,False,...,0.984583,0.986569,1.013045,0.987012,1.000000,1.01,0.99,0.001013,1.0,0.000909
3802,False,True,False,False,True,True,False,True,False,False,...,0.997881,1.338065,1.124255,0.882422,0.980000,1.01,0.99,0.001013,1.0,0.001321
3803,False,True,False,False,True,True,False,True,False,False,...,0.976473,0.913764,1.053832,0.947131,1.000000,1.01,0.99,0.001013,1.0,0.000838


In [5]:
for SR in df.columns[df.columns.str.contains("is_cut_sr_")]:
    print(f"{SR}: {df[df[SR]].evt_wgt_nominal.sum()}")

print("")
    
for SR in df.columns[df.columns.str.contains("is_bdt_sr_")]:
    print(f"{SR}: {df[df[SR]].evt_wgt_nominal.sum()}")

is_cut_sr_el: 0.3899495303630829
is_cut_sr_mu: 0.7241592407226562
is_cut_sr_tau: 0.26634103059768677
is_cut_sr_neg: 0.1924811601638794
is_cut_sr_lgt: 0.30952298641204834

is_bdt_sr_el: 0.6406623125076294
is_bdt_sr_mu: 1.2026352882385254
is_bdt_sr_tau: 0.3844670057296753
is_bdt_sr_neg: 0.3295164108276367


In [6]:
cut_based_table = pd.DataFrame()
bdt_based_table = pd.DataFrame()

# b-tagging scale factor variations

In [7]:
systs = make_syst_table(
    df, nominal_wgt="evt_wgt_nominal", wgt_col_pattern="btagsf_", sr_col_pattern="is_cut_sr_"
)
display(systs)
cut_based_table = append_systs(cut_based_table, systs)

systematics,cut_sr_el,cut_sr_lgt,cut_sr_mu,cut_sr_neg,cut_sr_tau
btagsf,4.6,3.5,4.6,4.6,4.9


In [8]:
systs = make_syst_table(
    df, nominal_wgt="evt_wgt_nominal", wgt_col_pattern="btagsf_", sr_col_pattern="is_bdt_sr_"
)
display(systs)
bdt_based_table = append_systs(bdt_based_table, systs)

systematics,bdt_sr_el,bdt_sr_mu,bdt_sr_neg,bdt_sr_tau
btagsf,4.9,5.2,4.9,4.5


# Jet energy scale variations

In [9]:
if JEC_UP_SAMPLE and JEC_DN_SAMPLE:
    with uproot.open(JEC_UP_SAMPLE) as f:
        df_up = f.get("variable").pandas.df(
            branches=["is_ps*", "is_bdt_*", "is_cut_*", "LHE_*", "pu_rewgt*",
                      "lepsf*", "btagsf*", "trigsf*", "wgt", "genrewgt"]
        )
    # Compute nominal event weight
    df_up["evt_wgt_nominal"] = df_up.wgt*df_up.genrewgt*df_up.lepsf*df_up.btagsf*df_up.trigsf*df_up.pu_rewgt
    df_up[df_up.columns[df_up.columns.str.contains("is_")]] = df_up[df_up.columns[df_up.columns.str.contains("is_")]].astype(bool)

    with uproot.open(JEC_DN_SAMPLE) as f:
        df_dn = f.get("variable").pandas.df(
            branches=["is_ps*", "is_bdt_*", "is_cut_*", "LHE_*", "pu_rewgt*",
                      "lepsf*", "btagsf*", "trigsf*", "wgt", "genrewgt"]
        )
    # Compute nominal event weight
    df_dn["evt_wgt_nominal"] = df_dn.wgt*df_dn.genrewgt*df_dn.lepsf*df_dn.btagsf*df_dn.trigsf*df_dn.pu_rewgt
    df_dn[df_dn.columns[df_dn.columns.str.contains("is_")]] = df_dn[df_dn.columns[df_dn.columns.str.contains("is_")]].astype(bool)

In [10]:
if JEC_UP_SAMPLE and JEC_DN_SAMPLE:
    # Compute all systematics
    rows = []
    for SR in df.columns[df.columns.str.contains("is_cut_sr_")]:
        n_pass_jec_up = df_up[df_up[SR]].evt_wgt_nominal.sum()
        n_pass_jec_dn = df_dn[df_dn[SR]].evt_wgt_nominal.sum()
        n_pass_nom = df[df[SR]].evt_wgt_nominal.sum()
        rows.append({
            "region":SR.split("is_")[-1],
            "wgt_name": "jec_up",
            "n_pass_wgt": n_pass_jec_up,
            "n_pass_nom": n_pass_nom,
            "delta_percent": (1 - n_pass_jec_up/(n_pass_nom))*100
        })
        rows.append({
            "region":SR.split("is_")[-1],
            "wgt_name": "jec_dn",
            "n_pass_wgt": n_pass_jec_dn,
            "n_pass_nom": n_pass_nom,
            "delta_percent": (1 - n_pass_jec_dn/(n_pass_nom))*100
        })

    systs = pd.DataFrame(data=rows)
    # Designate how columns should be renamed
    renames = {
        "delta_percent": "jec", 
        "region": "systematics"
    }
    # Return only one row: max systematic for each SR
    row = (systs.groupby("region", as_index=False)[["delta_percent"]]
                .agg(func=lambda x: np.max(np.abs(x)))
                .round(1)
                .rename(columns=renames)
                .set_index("systematics")
                .transpose())
    display(row)

    cut_based_table = append_systs(cut_based_table, row)

systematics,cut_sr_el,cut_sr_lgt,cut_sr_mu,cut_sr_neg,cut_sr_tau
jec,8.9,7.2,8.0,10.2,8.3


In [11]:
if JEC_UP_SAMPLE and JEC_DN_SAMPLE:
    # Compute all systematics
    rows = []
    for SR in df.columns[df.columns.str.contains("is_bdt_sr_")]:
        n_pass_jec_up = df_up[df_up[SR]].evt_wgt_nominal.sum()
        n_pass_jec_dn = df_dn[df_dn[SR]].evt_wgt_nominal.sum()
        n_pass_nom = df[df[SR]].evt_wgt_nominal.sum()
        rows.append({
            "region":SR.split("is_")[-1],
            "wgt_name": "jec_up",
            "n_pass_wgt": n_pass_jec_up,
            "n_pass_nom": n_pass_nom,
            "delta_percent": (1 - n_pass_jec_up/(n_pass_nom))*100
        })
        rows.append({
            "region":SR.split("is_")[-1],
            "wgt_name": "jec_dn",
            "n_pass_wgt": n_pass_jec_dn,
            "n_pass_nom": n_pass_nom,
            "delta_percent": (1 - n_pass_jec_dn/(n_pass_nom))*100
        })

    systs = pd.DataFrame(data=rows)
    # Designate how columns should be renamed
    renames = {
        "delta_percent": "jec", 
        "region": "systematics"
    }
    # Return only one row: max systematic for each SR
    row = (systs.groupby("region", as_index=False)[["delta_percent"]]
                .agg(func=lambda x: np.max(np.abs(x)))
                .round(1)
                .rename(columns=renames)
                .set_index("systematics")
                .transpose())
    display(row)

    bdt_based_table = append_systs(bdt_based_table, row)

systematics,bdt_sr_el,bdt_sr_mu,bdt_sr_neg,bdt_sr_tau
jec,10.4,8.4,9.9,7.9


# PU reweighting

In [12]:
systs = make_syst_table(
    df, nominal_wgt="evt_wgt_nominal", wgt_col_pattern="pu_rewgt_", sr_col_pattern="is_cut_sr_"
)
display(systs)
cut_based_table = append_systs(cut_based_table, systs)

systematics,cut_sr_el,cut_sr_lgt,cut_sr_mu,cut_sr_neg,cut_sr_tau
purewgt,0.8,1.2,0.5,2.3,1.2


In [13]:
systs = make_syst_table(
    df, nominal_wgt="evt_wgt_nominal", wgt_col_pattern="pu_rewgt_", sr_col_pattern="is_bdt_sr_"
)
display(systs)
bdt_based_table = append_systs(bdt_based_table, systs)

systematics,bdt_sr_el,bdt_sr_mu,bdt_sr_neg,bdt_sr_tau
purewgt,0.0,0.2,1.8,1.2


# LHE scale weight variations

In [14]:
# LHE_scale_wgt_sums = {}
# with open("LHE_scale_wgt_sums.txt", "r") as f_in:
#     for line in f_in.readlines():
#         name, value = line.split()
#         LHE_scale_wgt_sums[name] = float(value)
        
# NOMINAL_N_TOTAL = LHE_scale_wgt_sums["LHE_muF1p0_muR0p5"] # mu_R does not effect our signal sample

# print(LHE_scale_wgt_sums)

In [15]:
systs = make_syst_table(
    df, nominal_wgt="evt_wgt_nominal", wgt_col_pattern="LHE_mu", sr_col_pattern="is_cut_sr_"
)
display(systs)
cut_based_table = append_systs(cut_based_table, systs)

systematics,cut_sr_el,cut_sr_lgt,cut_sr_mu,cut_sr_neg,cut_sr_tau
LHEmu,22.5,17.9,22.3,24.3,22.4


In [16]:
systs = make_syst_table(
    df, nominal_wgt="evt_wgt_nominal", wgt_col_pattern="LHE_mu", sr_col_pattern="is_bdt_sr_"
)
display(systs)
bdt_based_table = append_systs(bdt_based_table, systs)

systematics,bdt_sr_el,bdt_sr_mu,bdt_sr_neg,bdt_sr_tau
LHEmu,21.4,21.6,23.1,21.2


# PDF variations

In [17]:
# LHE_pdf_wgt_sums = {}
# with open("LHE_pdf_wgt_sums.txt", "r") as f_in:
#     print(line)
#     for line in f_in.readlines():
#         name, value = line.split()
#         LHE_pdf_wgt_sums[name] = float(value)

# print(LHE_pdf_wgt_sums)

In [18]:
systs = make_syst_table(
    df, nominal_wgt="evt_wgt_nominal", wgt_col_pattern="LHE_pdf_wgt", sr_col_pattern="is_cut_sr_"
)
display(systs)
cut_based_table = append_systs(cut_based_table, systs)

systematics,cut_sr_el,cut_sr_lgt,cut_sr_mu,cut_sr_neg,cut_sr_tau
LHEpdfwgt,1.8,1.4,2.3,8.2,2.7


In [19]:
systs = make_syst_table(
    df, nominal_wgt="evt_wgt_nominal", wgt_col_pattern="LHE_pdf_wgt", sr_col_pattern="is_bdt_sr_"
)
display(systs)
bdt_based_table = append_systs(bdt_based_table, systs)

systematics,bdt_sr_el,bdt_sr_mu,bdt_sr_neg,bdt_sr_tau
LHEpdfwgt,1.7,2.4,7.1,2.6


# Trigger scale factor variations

In [20]:
systs = make_syst_table(
    df, nominal_wgt="evt_wgt_nominal", wgt_col_pattern="trigsf", sr_col_pattern="is_cut_sr_"
)
display(systs)
cut_based_table = append_systs(cut_based_table, systs)

systematics,cut_sr_el,cut_sr_lgt,cut_sr_mu,cut_sr_neg,cut_sr_tau
trigsf,1.0,1.0,1.3,1.7,5.0


In [21]:
systs = make_syst_table(
    df, nominal_wgt="evt_wgt_nominal", wgt_col_pattern="trigsf", sr_col_pattern="is_bdt_sr_"
)
display(systs)
bdt_based_table = append_systs(bdt_based_table, systs)

systematics,bdt_sr_el,bdt_sr_mu,bdt_sr_neg,bdt_sr_tau
trigsf,1.0,1.3,1.6,5.0


# Lepton ID scale factor variations

In [22]:
systs = make_syst_table(
    df, nominal_wgt="evt_wgt_nominal", wgt_col_pattern="lepsf", sr_col_pattern="is_cut_sr_"
)
display(systs)
cut_based_table = append_systs(cut_based_table, systs)

systematics,cut_sr_el,cut_sr_lgt,cut_sr_mu,cut_sr_neg,cut_sr_tau
lepsf,12.4,6.8,3.7,5.5,5.4


In [23]:
systs = make_syst_table(
    df, nominal_wgt="evt_wgt_nominal", wgt_col_pattern="lepsf", sr_col_pattern="is_bdt_sr_"
)
display(systs)
bdt_based_table = append_systs(bdt_based_table, systs)

systematics,bdt_sr_el,bdt_sr_mu,bdt_sr_neg,bdt_sr_tau
lepsf,12.5,3.7,5.6,5.4


# Statistical uncertainty

In [24]:
with uproot.open(RUN2_NOMINAL_SAMPLE) as f:
    run2_df = f.get("variable").pandas.df(
        branches=["is_bdt_*", "is_cut_*", "pu_rewgt", "lepsf", "btagsf", "trigsf", "wgt", "genrewgt"]
    )
# Compute nominal event weight
run2_df["evt_wgt_nominal"] = run2_df.wgt*run2_df.genrewgt*run2_df.lepsf*run2_df.btagsf*run2_df.trigsf*run2_df.pu_rewgt

run2_df[run2_df.columns[run2_df.columns.str.contains("is_")]] = run2_df[run2_df.columns[run2_df.columns.str.contains("is_")]].astype(bool)

run2_df

,is_bdt_sr_el,is_bdt_sr_mu,is_bdt_sr_tau,is_bdt_sr_neg,is_bdt_cr_el,is_bdt_cr_mu,is_bdt_cr_tau,is_bdt_cr_neg,is_bdt_sr2_el,is_bdt_sr2_mu,...,is_cut_cr,is_cut_sr2,is_cut_cr2,pu_rewgt,lepsf,btagsf,trigsf,wgt,genrewgt,evt_wgt_nominal
entry,,,,,,,,,,,,,,,,,,,,,
0,False,False,False,False,True,False,False,False,False,False,...,True,False,True,1.051170,0.832435,1.052887,1.00,0.001329,1.0,0.001224
1,False,False,False,False,False,False,False,False,False,False,...,False,True,False,1.182680,0.760838,1.013416,1.00,0.001329,1.0,0.001212
2,False,False,False,False,False,False,False,False,False,False,...,False,True,False,1.049860,0.761182,0.881712,1.00,0.001329,1.0,0.000936
3,False,True,False,False,False,False,False,False,False,True,...,False,True,False,1.058170,1.008093,0.917499,0.99,0.001329,1.0,0.001287
4,False,True,False,False,False,False,False,False,False,True,...,False,True,False,1.072160,1.001989,0.985739,0.99,0.001329,1.0,0.001393
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8403,False,False,False,False,False,False,False,False,False,True,...,False,False,False,1.003390,0.906175,0.986569,1.00,0.001013,1.0,0.000909
8404,False,True,False,False,False,False,False,False,False,True,...,True,False,True,0.992600,1.001997,1.338065,0.98,0.001013,1.0,0.001321
8405,False,True,False,False,False,False,False,False,False,True,...,False,True,False,0.992600,0.911827,0.913764,1.00,0.001013,1.0,0.000838


In [25]:
for SR in run2_df.columns[run2_df.columns.str.contains("is_cut_sr_")]:
    count = run2_df[run2_df[SR]].evt_wgt_nominal.sum()
    error = np.sqrt(np.sum(run2_df[run2_df[SR]].evt_wgt_nominal**2))
    print(f"{SR}: {count} +- {error}")

print("")
    
for SR in run2_df.columns[run2_df.columns.str.contains("is_bdt_sr_")]:
    count = run2_df[run2_df[SR]].evt_wgt_nominal.sum()
    error = np.sqrt(np.sum(run2_df[run2_df[SR]].evt_wgt_nominal**2))
    print(f"{SR}: {count} +- {error}")

is_cut_sr_el: 0.8637039661407471 +- 0.027406960725784302
is_cut_sr_mu: 1.658313512802124 +- 0.040238093584775925
is_cut_sr_tau: 0.4374115467071533 +- 0.019122833386063576
is_cut_sr_neg: 0.4238339066505432 +- 0.0196287352591753
is_cut_sr_lgt: 0.6580057144165039 +- 0.024757491424679756

is_bdt_sr_el: 1.3904001712799072 +- 0.035136785358190536
is_bdt_sr_mu: 2.7156310081481934 +- 0.052107807248830795
is_bdt_sr_tau: 0.6356835961341858 +- 0.02299283631145954
is_bdt_sr_neg: 0.695464015007019 +- 0.025460602715611458


In [26]:
def get_stats(df, sr_col_pattern, nom_wgt):
    rows = []
    for SR in df.columns[df.columns.str.contains(sr_col_pattern)]:
        in_SR = df[SR]
        count = np.sum(df[in_SR][nom_wgt])
        error = np.sqrt(np.sum(df[in_SR][nom_wgt]**2))
        rows.append({
            "region":SR.split("is_")[-1],
            "count": count,
            "error": error,
            "delta_percent": (error/count)*100
        })
            
    return pd.DataFrame(data=rows)

def make_stat_table(df, sr_col_pattern, nominal_wgt="", debug=False):
    # Compute all systematics
    stats = get_stats(df, sr_col_pattern, nominal_wgt)
    if debug:
        display(stats)
    # Designate how columns should be renamed
    renames = {
        "delta_percent": "statunc", 
        "region": "systematics"
    }
    # Return only one row: max systematic for each SR
    return (stats.groupby("region", as_index=False)[["delta_percent"]]
                 .agg(func=lambda x: np.max(np.abs(x)))
                 .round(1)
                 .rename(columns=renames)
                 .set_index("systematics")
                 .transpose())

In [27]:
stats = make_stat_table(run2_df, "is_cut_sr_", nominal_wgt="evt_wgt_nominal", debug=True)
display(stats)
cut_based_table = append_systs(cut_based_table, stats)

,region,count,error,delta_percent
0,cut_sr_el,0.863704,0.027407,3.173189
1,cut_sr_mu,1.658314,0.040238,2.426447
2,cut_sr_tau,0.437412,0.019123,4.371817
3,cut_sr_neg,0.423834,0.019629,4.631233
4,cut_sr_lgt,0.658006,0.024757,3.762504


systematics,cut_sr_el,cut_sr_lgt,cut_sr_mu,cut_sr_neg,cut_sr_tau
statunc,3.2,3.8,2.4,4.6,4.4


In [28]:
stats = make_stat_table(run2_df, "is_bdt_sr_", nominal_wgt="evt_wgt_nominal", debug=True)
display(stats)
bdt_based_table = append_systs(bdt_based_table, stats)

,region,count,error,delta_percent
0,bdt_sr_el,1.390400,0.035137,2.527099
1,bdt_sr_mu,2.715631,0.052108,1.918810
2,bdt_sr_tau,0.635684,0.022993,3.617025
3,bdt_sr_neg,0.695464,0.025461,3.660952


systematics,bdt_sr_el,bdt_sr_mu,bdt_sr_neg,bdt_sr_tau
statunc,2.5,1.9,3.7,3.6


# Final tables

In [29]:
cut_based_table = cut_based_table[["cut_sr_el", "cut_sr_mu", "cut_sr_tau", "cut_sr_neg", "cut_sr_lgt"]]
display(cut_based_table)
print(cut_based_table.to_latex())

systematics,cut_sr_el,cut_sr_mu,cut_sr_tau,cut_sr_neg,cut_sr_lgt
btagsf,4.6,4.6,4.9,4.6,3.5
jec,8.9,8.0,8.3,10.2,7.2
purewgt,0.8,0.5,1.2,2.3,1.2
LHEmu,22.5,22.3,22.4,24.3,17.9
LHEpdfwgt,1.8,2.3,2.7,8.2,1.4
trigsf,1.0,1.3,5.0,1.7,1.0
lepsf,12.4,3.7,5.4,5.5,6.8
statunc,3.2,2.4,4.4,4.6,3.8


\begin{tabular}{lrrrrr}
\toprule
systematics &  cut\_sr\_el &  cut\_sr\_mu &  cut\_sr\_tau &  cut\_sr\_neg &  cut\_sr\_lgt \\
\midrule
btagsf    &        4.6 &        4.6 &         4.9 &         4.6 &         3.5 \\
jec       &        8.9 &        8.0 &         8.3 &        10.2 &         7.2 \\
purewgt   &        0.8 &        0.5 &         1.2 &         2.3 &         1.2 \\
LHEmu     &       22.5 &       22.3 &        22.4 &        24.3 &        17.9 \\
LHEpdfwgt &        1.8 &        2.3 &         2.7 &         8.2 &         1.4 \\
trigsf    &        1.0 &        1.3 &         5.0 &         1.7 &         1.0 \\
lepsf     &       12.4 &        3.7 &         5.4 &         5.5 &         6.8 \\
statunc   &        3.2 &        2.4 &         4.4 &         4.6 &         3.8 \\
\bottomrule
\end{tabular}



In [30]:
aggs = cut_based_table.transpose().agg(["min", "max"]).transpose()
aggs["min_to_max"] = aggs["min"].astype(str) + "%-" + aggs["max"].astype(str) + "%"
print(aggs[["min_to_max"]].to_latex())

\begin{tabular}{ll}
\toprule
{} &   min\_to\_max \\
\midrule
btagsf    &    3.5\%-4.9\% \\
jec       &   7.2\%-10.2\% \\
purewgt   &    0.5\%-2.3\% \\
LHEmu     &  17.9\%-24.3\% \\
LHEpdfwgt &    1.4\%-8.2\% \\
trigsf    &    1.0\%-5.0\% \\
lepsf     &   3.7\%-12.4\% \\
statunc   &    2.4\%-4.6\% \\
\bottomrule
\end{tabular}



In [31]:
bdt_based_table = bdt_based_table[["bdt_sr_el", "bdt_sr_mu", "bdt_sr_tau", "bdt_sr_neg"]]
display(bdt_based_table)
print(bdt_based_table.to_latex())

systematics,bdt_sr_el,bdt_sr_mu,bdt_sr_tau,bdt_sr_neg
btagsf,4.9,5.2,4.5,4.9
jec,10.4,8.4,7.9,9.9
purewgt,0.0,0.2,1.2,1.8
LHEmu,21.4,21.6,21.2,23.1
LHEpdfwgt,1.7,2.4,2.6,7.1
trigsf,1.0,1.3,5.0,1.6
lepsf,12.5,3.7,5.4,5.6
statunc,2.5,1.9,3.6,3.7


\begin{tabular}{lrrrr}
\toprule
systematics &  bdt\_sr\_el &  bdt\_sr\_mu &  bdt\_sr\_tau &  bdt\_sr\_neg \\
\midrule
btagsf    &        4.9 &        5.2 &         4.5 &         4.9 \\
jec       &       10.4 &        8.4 &         7.9 &         9.9 \\
purewgt   &        0.0 &        0.2 &         1.2 &         1.8 \\
LHEmu     &       21.4 &       21.6 &        21.2 &        23.1 \\
LHEpdfwgt &        1.7 &        2.4 &         2.6 &         7.1 \\
trigsf    &        1.0 &        1.3 &         5.0 &         1.6 \\
lepsf     &       12.5 &        3.7 &         5.4 &         5.6 \\
statunc   &        2.5 &        1.9 &         3.6 &         3.7 \\
\bottomrule
\end{tabular}



In [32]:
aggs = bdt_based_table.transpose().agg(["min", "max"]).transpose()
aggs["min_to_max"] = aggs["min"].astype(str) + "%-" + aggs["max"].astype(str) + "%"
print(aggs[["min_to_max"]].to_latex())

\begin{tabular}{ll}
\toprule
{} &   min\_to\_max \\
\midrule
btagsf    &    4.5\%-5.2\% \\
jec       &   7.9\%-10.4\% \\
purewgt   &    0.0\%-1.8\% \\
LHEmu     &  21.2\%-23.1\% \\
LHEpdfwgt &    1.7\%-7.1\% \\
trigsf    &    1.0\%-5.0\% \\
lepsf     &   3.7\%-12.5\% \\
statunc   &    1.9\%-3.7\% \\
\bottomrule
\end{tabular}

